In [1]:
%pwd

'd:\\Capestone projects\\Project_2\\document_summarization_for_legal_texts\\research'

In [2]:
import os
os.chdir("../")
%pwd

'd:\\Capestone projects\\Project_2\\document_summarization_for_legal_texts'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [4]:
from datascience.constants import *
from datascience.utils.common import read_yaml, create_directories

In [5]:
class configuration_manager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [6]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\Admin\anaconda3\envs\summary\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-07-06 13:00:09,448] -> [INFO]: config - PyTorch version 2.7.1+cu118 available.


In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        
        #os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Optional: disables GPU usage

        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_t5_small = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5_small)
        
        # loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(
            model=model_t5_small, args=trainer_args,
            tokenizer=tokenizer, data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"]
        )
        
        trainer.train()
        
        # Save model and tokenizer
        model_t5_small.save_pretrained(os.path.join(self.config.root_dir, "t5-legal-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [8]:
try:
    config = configuration_manager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-07-06 13:00:10,039] -> [INFO]: common - yaml file: config\config.yaml loaded successfully
[2025-07-06 13:00:10,049] -> [INFO]: common - yaml file: params.yaml loaded successfully
[2025-07-06 13:00:10,051] -> [INFO]: common - created directory at: artifacts
[2025-07-06 13:00:10,053] -> [INFO]: common - created directory at: artifacts/model_trainer


C:\Users\Admin\AppData\Local\Temp\ipykernel_26032\1964146882.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,4.752800
20,4.670800
30,4.627600
40,4.592000
50,4.513600
60,4.286000
70,4.122600
80,3.966700
90,3.676500
100,3.454100
